In [22]:
from modules import *
# baseline_history = load_data_and_train_model()
from submodules import train_model

def model_builder(hp):
    import tensorflow as tf
    from tensorflow import keras
    model = tf.keras.Sequential()


    # Tune the number of units in the first Dense layer
    # Choose an optimal value between 32-512
    hp_units = hp.Int('units', min_value=256, max_value=2048, step=32)
    model.add(keras.layers.Dense(units=hp_units, activation='relu',input_shape=(x_train.shape[-1],)))
    model.add(keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(3, activation='softmax'))

    # Tune the learning rate for the optimizer
    # Choose an optimal value from 0.01, 0.001, or 0.0001
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
                metrics=['accuracy'
        # keras.metrics.CategoricalAccuracy(name='categorical_accuracy'),
        # keras.metrics.Precision(name='precision'),
        # keras.metrics.Recall(name='recall'),
        # keras.metrics.AUC(name='auc')
    ])
    return model
import keras_tuner as kt
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

INFO:tensorflow:Reloading Oracle from existing project my_dir/intro_to_kt/oracle.json
INFO:tensorflow:Reloading Tuner from my_dir/intro_to_kt/tuner0.json


In [14]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
train_df = pd.read_csv("train.csv")
# val_df = pd.read_csv("val.csv")
y_train = train_df.pop('Class')
x_train = train_df
# y_val = val_df.pop('Class')
# x_val = val_df
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
# x_val = scaler.fit_transform(x_val)
x_train = np.array(x_train)
y_train = np.array(y_train)
# x_val = np.array(x_val)
# y_val = np.array(y_val)
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
tuner.search(x_train, tf.one_hot(y_train, 3), epochs=50, validation_split=0.2, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

Trial 31 Complete [00h 00m 22s]
val_accuracy: 0.8901441097259521

Best val_accuracy So Far: 0.8938691020011902
Total elapsed time: 00h 06m 24s
INFO:tensorflow:Oracle triggered exit

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 320 and the optimal learning rate for the optimizer
is 0.01.



In [17]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(x_train, tf.one_hot(y_train, 3), epochs=500, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/500
2047/2047 [==============================] - 2s 993us/step - loss: 0.4009 - accuracy: 0.8529 - val_loss: 0.3415 - val_accuracy: 0.8792
Epoch 2/500
2047/2047 [==============================] - 2s 854us/step - loss: 0.3667 - accuracy: 0.8704 - val_loss: 0.3430 - val_accuracy: 0.8779
Epoch 3/500
2047/2047 [==============================] - 2s 868us/step - loss: 0.3570 - accuracy: 0.8728 - val_loss: 0.3299 - val_accuracy: 0.8837
Epoch 4/500
2047/2047 [==============================] - 2s 843us/step - loss: 0.3532 - accuracy: 0.8735 - val_loss: 0.3188 - val_accuracy: 0.8879
Epoch 5/500
2047/2047 [==============================] - 2s 852us/step - loss: 0.3482 - accuracy: 0.8771 - val_loss: 0.3274 - val_accuracy: 0.8846
Epoch 6/500
2047/2047 [==============================] - 2s 935us/step - loss: 0.3470 - accuracy: 0.8780 - val_loss: 0.3176 - val_accuracy: 0.8867
Epoch 7/500
2047/2047 [==============================] - 2s 937us/step - loss: 0.3430 - accuracy: 0.8805 - val_loss: 0

In [18]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(x_train, tf.one_hot(y_train, 3), epochs=best_epoch, validation_split=0.2)

Epoch 1/414
2047/2047 [==============================] - 2s 988us/step - loss: 0.4020 - accuracy: 0.8528 - val_loss: 0.3379 - val_accuracy: 0.8806
Epoch 2/414
2047/2047 [==============================] - 2s 889us/step - loss: 0.3659 - accuracy: 0.8702 - val_loss: 0.3231 - val_accuracy: 0.8859
Epoch 3/414
2047/2047 [==============================] - 2s 1ms/step - loss: 0.3580 - accuracy: 0.8730 - val_loss: 0.3251 - val_accuracy: 0.8844
Epoch 4/414
2047/2047 [==============================] - 2s 870us/step - loss: 0.3518 - accuracy: 0.8767 - val_loss: 0.3276 - val_accuracy: 0.8834
Epoch 5/414
2047/2047 [==============================] - 2s 866us/step - loss: 0.3514 - accuracy: 0.8761 - val_loss: 0.3128 - val_accuracy: 0.8881
Epoch 6/414
2047/2047 [==============================] - 2s 882us/step - loss: 0.3461 - accuracy: 0.8785 - val_loss: 0.3259 - val_accuracy: 0.8868
Epoch 7/414
2047/2047 [==============================] - 2s 827us/step - loss: 0.3441 - accuracy: 0.8797 - val_loss: 0.3

In [21]:
from submodules import test_model,plot_cm
from tensorflow import one_hot
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
test_df = pd.read_csv("test.csv")
y_test = test_df.pop('Class')
x_test = test_df
x_test = np.array(x_test)
y_test = np.array(y_test)
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
x_test = scaler.fit_transform(x_test)
eval_result = hypermodel.evaluate(x_test, tf.one_hot(y_test,3))
print("[test loss, test accuracy]:", eval_result)

640/640 [==============================] - 0s 435us/step - loss: 0.7544 - accuracy: 0.8633
[test loss, test accuracy]: [0.7543939352035522, 0.8632633090019226]
